<a href="https://colab.research.google.com/github/CarlosMendez1997Col/Automation_of_satellite_image_downloads_GeeMap_APIs_REST/blob/main/1.%20Sentinel/Sentinel1_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Automated script to download Sentinel-1 images from a Google Earth Engine using GeeMap`



# Import libraries and packages

In [1]:
!pip install geemap # Install the geemap library for interactive mapping with Earth Engine
!pip install earthengine-api # Install the Earth Engine Python API

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.9 MB/s eta 0:00:00


In [5]:
import ee # Import the Earth Engine API
import geemap # Import geemap for visualization and mapping
import requests
import os
import shutil
import geopandas as gpd
from google.colab import files
import webbrowser # Import webbrowser to open authentication URLs
import ipywidgets as widgets # Import ipywidgets for interactive widgets in Jupyter notebooks
from datetime import datetime # Import datetime for handling dates

## Autentication in Google Colab and GEE

In [3]:
# --- Authenticate and initialize Earth Engine ---
auth_url = ee.Authenticate(auth_mode='notebook') # Generate authentication URL for notebook mode
webbrowser.open(auth_url) # Open the authentication URL in the browser
ee.Authenticate() # Perform authentication
ee.Initialize() # Initialize the Earth Engine session

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/drive%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=OqE8f94F3oRjgNuTk2ftF1nafM75Sq8qE8CWv3QCRKE&tc=flv1DiHCo05AJXM1Zvz5IVy1VEC6iqMvxK_tqz3-H88&cc=QL67IgBYXWI0Yfix5Az_aPhpK_RcPSerMm5PB6S82WA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ASc3gC37SosvQ9JD_IDf21CsXyRC7_fTrWuzyuAVjvWwzyk76XPlyRMWqcQ

Successfully saved authorization token.


## Create basemaps and import Area Of Interest (AOI)

In [4]:
# --- Load Cobija shapefile from Earth Engine assets ---
cobija_fc = ee.FeatureCollection("projects/gee-projects-481514/assets/CobijaSHP")

# --- Create a map and add Cobija shapefile ---
MapCobija = geemap.Map() # Create a geemap Map object
MapCobija.addLayer(cobija_fc, {"color": "darkblue", "opacity": 0.7}, "Cobija SHP") # Add Cobija shapefile layer
MapCobija.centerObject(cobija_fc, 13) # Center the map on Cobija with zoom level 13
MapCobija.setOptions('HYBRID') # Set basemap style to hybrid (satellite + labels)

# --- Define available basemap styles ---
styles = {
    "OpenStreetMap": "OpenStreetMap",
    "Esri World Street Map": "Esri.WorldStreetMap",
    "Esri World Imagery": "Esri.WorldImagery",
    "Esri World Topo Map": "Esri.WorldTopoMap",
    "Esri World Gray Canvas": "Esri.WorldGrayCanvas",
    "Esri World Shaded Relief": "Esri.WorldShadedRelief",
    "Esri World Terrain": "Esri.WorldTerrain",
    "CartoDB DarkMatter": "CartoDB.DarkMatter",
    "CartoDB Positron": "CartoDB.Positron",
    "CartoDB Voyager": "CartoDB.Voyager",
    "OpenTopoMap": "OpenTopoMap"
}

# --- Create dropdown widget to select basemap style ---
style_dropdown = widgets.Dropdown(
    options=list(styles.keys()), # Options are the keys of the styles dictionary
    value="Esri World Imagery", # Default selected basemap
    description="Basemap:" # Label for the dropdown
)

# --- Function to update basemap style when dropdown changes ---
def update_style(change):
    MapCobija.add_basemap(styles[change['new']]) # Add new basemap selected by user
    MapCobija.addLayer(cobija_fc, {"color": "darkblue", "opacity": 0.7}, "Cobija SHP") # Re-add Cobija layer
    MapCobija.centerObject(cobija_fc, 13) # Re-center map on Cobija

style_dropdown.observe(update_style, names='value') # Observe changes in dropdown value

display(style_dropdown) # Display the dropdown widget
MapCobija # Display the map

Dropdown(description='Basemap:', index=2, options=('OpenStreetMap', 'Esri World Street Map', 'Esri World Image…

Map(center=[-11.031617083854373, -68.77127014491582], controls=(WidgetControl(options=['position', 'transparen…

# Search Sentinel Images in the AOI

In [6]:
# --- Subir shapefile ---
uploaded = files.upload()  # selecciona .shp, .shx, .dbf, .prj

# --- Leer shapefile y reproyectar ---
shp_path = "/content/CobijaAOI.shp"
gdf = gpd.read_file(shp_path).to_crs(epsg=4326)

# --- Calcular bounding box ---
xmin, ymin, xmax, ymax = gdf.total_bounds

Saving CobijaAOI.CPG to CobijaAOI.CPG
Saving CobijaAOI.dbf to CobijaAOI.dbf
Saving CobijaAOI.prj to CobijaAOI.prj
Saving CobijaAOI.sbn to CobijaAOI.sbn
Saving CobijaAOI.sbx to CobijaAOI.sbx
Saving CobijaAOI.shp to CobijaAOI.shp
Saving CobijaAOI.shx to CobijaAOI.shx


## Sentinel-1 SAR GRD: C-band Synthetic Aperture Radar Ground Range Detected

In [7]:
bbox = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax], proj='EPSG:4326', geodesic=False)

# --- Load Sentinel-1 GRD image collection ---
s1_collection = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(bbox) # Filter by Cobija region

# --- Get start and end dates for filtering ---
first_image = s1_collection.sort('system:time_start').first() # Get earliest image
start_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo() # Extract start date
end_date = datetime.utcnow().strftime("%Y-%m-%d") # Use current date as end date

# --- Filter Sentinel-1 collection by parameters ---
s1 = (s1_collection
      .filterDate(start_date, end_date) # Filter by date range
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) # Require VV polarization
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) # Require VH polarization
      .filter(ee.Filter.eq('instrumentMode', 'IW')) # Use Interferometric Wide swath mode
      .filter(ee.Filter.eq('productType', 'GRD')) # Use GRD product type
      .filter(ee.Filter.eq('platform_number', 'A')) # Use Sentinel-1A platform
      .sort('system:time_start', False)) # Sort by date descending

# --- Additional orbit filters ---
orbit_filter_pass = 'DESCENDING' # Orbit direction filter
relative_orbit = 127 # Relative orbit number filter

if orbit_filter_pass is not None:
    s1 = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit_filter_pass)) # Apply orbit pass filter

if relative_orbit is not None:
    s1 = s1.filter(ee.Filter.eq('relativeOrbitNumber_start', relative_orbit)) # Apply relative orbit filter

## Additional processes in S1 Images


In [19]:
# --- Clip each image to bounding box ---
def clip_bbox(image):
    return image.clip(bbox)

# --- Implementación Gamma MAP ---
def gammamap(image, KERNEL_SIZE=7):
    enl = 5
    bandNames = image.bandNames().remove('angle')
    reducers = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
    )
    stats = image.select(bandNames).reduceNeighborhood(
        reducer=reducers,
        kernel=ee.Kernel.square(KERNEL_SIZE//2, 'pixels'),
        optimization='window'
    )
    meanBand = bandNames.map(lambda b: ee.String(b).cat('_mean'))
    stdDevBand = bandNames.map(lambda b: ee.String(b).cat('_stdDev'))

    z = stats.select(meanBand)
    sigz = stats.select(stdDevBand)
    ci = sigz.divide(z)

    cu = 1.0/(enl**0.5)
    cmax = (2**0.5) * cu

    cu = ee.Image.constant(cu)
    cmax = ee.Image.constant(cmax)
    enlImg = ee.Image.constant(enl)
    oneImg = ee.Image.constant(1)
    twoImg = ee.Image.constant(2)

    alpha = oneImg.add(cu.pow(2)).divide(ci.pow(2).subtract(cu.pow(2)))
    q = image.select(bandNames).expression(
        "z**2 * (z * alpha - enl - 1)**2 + 4 * alpha * enl * b() * z",
        {"z": z, "alpha": alpha, "enl": enl}
    )
    rHat = z.multiply(alpha.subtract(enlImg).subtract(oneImg)).add(q.sqrt()).divide(twoImg.multiply(alpha))

    zHat = z.updateMask(ci.lte(cu)).rename(bandNames)
    rHat = rHat.updateMask(ci.gt(cu)).updateMask(ci.lt(cmax)).rename(bandNames)
    x = image.select(bandNames).updateMask(ci.gte(cmax)).rename(bandNames)

    output = ee.ImageCollection([zHat, rHat, x]).sum()
    # Renombrar las bandas filtradas
    filteredBands = bandNames.map(lambda b: ee.String(b).cat('_filtered'))
    output = output.rename(filteredBands)

    return image.addBands(output)

## Apply additional processes

In [20]:
# --- Apply preprocessing functions to entire collection ---
s1_preprocessed = (s1.map(lambda img: gammamap(img, 7)) # Apply speckle filter
                     .map(clip_bbox)) # Clip to bounding box

## Print S1 images in AOI

In [16]:
# --- Create list of images and extract metadata ---
images_list = s1_preprocessed.toList(s1_preprocessed.size()) # Convert collection to list
info_list = []
for i in range(s1_preprocessed.size().getInfo()): # Loop through all images
    img = ee.Image(images_list.get(i)) # Get image
    info = img.getInfo() # Get image metadata
    info_list.append({
        'ID': info['id'], # Image ID
        'Fecha': info['properties']['system:time_start'], # Acquisition date
        'Polarización': info['properties']['transmitterReceiverPolarisation'] # Polarization info
    })

print("Imágenes Sentinel-1 preprocesadas disponibles en Cobija:")
for info in info_list:
    print(info) # Print metadata for each image

Imágenes Sentinel-1 preprocesadas disponibles en Cobija:
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20260108T101431_20260108T101501_062674_07DBA5_DF0D', 'Fecha': 1767867271000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251227T101432_20251227T101502_062499_07D4E7_D566', 'Fecha': 1766830472000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251215T101433_20251215T101503_062324_07CE1A_87D7', 'Fecha': 1765793673000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251203T101435_20251203T101504_062149_07C742_86A4', 'Fecha': 1764756875000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251121T101436_20251121T101506_061974_07C06E_317E', 'Fecha': 1763720076000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20251109T101436_20251109T101506_061799_07B9A0_2B70', 'Fecha': 1762683276000, 'Polarización': ['VV', 'VH']}
{'ID': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_

## Visualize latest 3 images of S1 in the AOI

In [21]:
# --- Limitar a las últimas 3 imágenes ---
s1_10 = s1_preprocessed.limit(3)
images_list_10 = s1_10.toList(s1_10.size())

# --- Crear mapa para visualización ---
MapCobijaS1 = geemap.Map()

for i in range(3):  # Solo 3 imágenes
    img = ee.Image(images_list_10.get(i))
    # Fecha como string server-side
    date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()

    MapCobijaS1.addLayer(img.select('VV'), {"min": -25, "max": 0}, f"VV {date_str}")
    MapCobijaS1.addLayer(img.select('VV_filtered'), {"min": -25, "max": 0}, f"VV filtered {date_str}")
    MapCobijaS1.addLayer(img.select('VH'), {"min": -30, "max": 0}, f"VH {date_str}")
    MapCobijaS1.addLayer(img.select('VH_filtered'), {"min": -30, "max": 0}, f"VH filtered {date_str}")

MapCobijaS1.addLayer(cobija_fc, {"color":"darkblue", "opacity":0.7}, "Cobija SHP")
MapCobijaS1.centerObject(cobija_fc, 12)
MapCobijaS1

Map(center=[-11.031617083854373, -68.77127014491582], controls=(WidgetControl(options=['position', 'transparen…

## Export S1 images to Google Colab Workspace

In [ ]:
# Create local folder in Colab workspace
os.makedirs("Sentinel1_Exports", exist_ok=True)

# Function to download VV and VH bands for each image
def download_images(collection, region):
    images_list = collection.toList(collection.size())
    n = collection.size().getInfo()

    for i in range(n):
        img = ee.Image(images_list.get(i))
        date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()

        # Export VV band
        if 'VV' in img.bandNames().getInfo():
            url_vv = img.select('VV').getDownloadURL({
                'scale': 10,
                'crs': 'EPSG:4326',
                'region': region
            })
            r = requests.get(url_vv)
            with open(f"Sentinel1_Exports/S1_VV_SF_gammamap_{date_str}.tif", 'wb') as f:
                f.write(r.content)

        # Export VH band
        if 'VH' in img.bandNames().getInfo():
            url_vh = img.select('VH').getDownloadURL({
                'scale': 10,
                'crs': 'EPSG:4326',
                'region': region
            })
            r = requests.get(url_vh)
            with open(f"Sentinel1_Exports/S1_VH_SF_gammamap_{date_str}.tif", 'wb') as f:
                f.write(r.content)

# Call function with your clipped collection and bounding box
download_images(s1_preprocessed, bbox)

## Download S1 images with VH and VV polarizations

In [ ]:
# Folder with the raster images
folder_path = "Sentinel1_Exports"

# Filename to compress
zip_name = "Sentinel1_Exports_ZIP"

# Folder and location of filename
shutil.make_archive(zip_name, 'zip', folder_path)

# Download S1 images
files.download(f"{zip_name}.zip")